# Setup

In [1]:
# import modules
import pandas as pd
import numpy as np
from  urllib2 import urlopen #to read in html
from bs4 import BeautifulSoup #for web scraping
import re

# for file directory
import os

%matplotlib inline

In [2]:
## project directory
folder = "C:\\Users\\lisa.hornung\\Documents\\Projects\\1812-drei-fragezeichen"

# Scrape data of websites

## Rocky-beach.com

### Setup list of web pages to scrape

In [3]:
# generate list with links to scrape
html_list = []
episode = ["%03d" % number for number in range(1, 205) if number != 29] #exclude 29 since it's a special edition
website = "https://www.rocky-beach.com/hoerspiel/folgen/"
for i in range(len(episode)):
    html = "%s%s%s" % (website, episode[i], ".html")
    html_list.append(html)

html_list[:4]

['https://www.rocky-beach.com/hoerspiel/folgen/001.html',
 'https://www.rocky-beach.com/hoerspiel/folgen/002.html',
 'https://www.rocky-beach.com/hoerspiel/folgen/003.html',
 'https://www.rocky-beach.com/hoerspiel/folgen/004.html']

### Meta data

#### Get tables to scrape

In [4]:
## get the table that contains the data we want for each site
meta_tables = [] #list to hold all indices for the correct table for each site
for a in range(len(html_list)):
    site=pd.read_html(html_list[a])
    table = []
    for i in range(len(site)):
        if site[i][0].dtype == "O": #test if the first column in table is object
            if site[i][0].str.encode("utf8").str.contains('Studio-Infos').any() == True: #check if contains keyword
                table.append(i) #create list that holds all the tables with keyword
    table=table[-1] + 1 #get last element and add one since it's the table we want
    meta_tables.append(table)
    
meta_tables[:4]

[7, 7, 7, 7]

#### Get the data

In [5]:
#Get meta data for all pages
meta = []

for i in range(len(html_list)):
    meta_site = pd.read_html(html_list[i], encoding="utf-8")[meta_tables[i]].dropna(0)#read in the table with the meta data for each site
    title = BeautifulSoup(urlopen(html_list[i]), 'lxml').title.string.encode('utf8').replace('(H\xc3\xb6rspiel)', '') #get title
    meta_site = meta_site.append({0: 'Titel:', 1: title}, ignore_index=True) #append title column
    meta_site = meta_site.append({0: 'html', 1: html_list[i]}, ignore_index=True) #add url
    meta_site[0] = meta_site[0].str.replace(":", "").str.strip() #format data
    meta_site["ID"] = episode[i] #add ID column
    meta.append(meta_site)

meta = pd.concat(meta, axis=0) #join all individual tables together
meta = meta.groupby(["ID", 0]).agg({1:"first"}).unstack().reset_index()
meta.columns = [col[1] for col in meta.columns]

meta.head()

,ID,"""Anudhara"" aufgenommen bei Bantree Records, Worms / Produzent",Buch,Buch und Effekte,Buch und Redaktion,Cover-Illustration,Coverillustration,Design,Effekte und Redaktion,Erscheinungsdatum,...,Redaktion und Geräusche,Regie,Regie und Produktion,Song,Teil A erzählt von,Teil B erzählt von,Teil C erzählt von,Titel,Titelmusik,html
0,001,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,NaN,Heikedine Körting,NaN,NaN,NaN,NaN,NaN,Der Super-Papagei,NaN,https://www.rocky-beach.com/hoerspiel/folgen/0...
1,002,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,NaN,Heikedine Körting,NaN,NaN,NaN,NaN,NaN,Der Phantomsee,NaN,https://www.rocky-beach.com/hoerspiel/folgen/0...
2,003,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,NaN,Heikedine Körting,NaN,NaN,NaN,NaN,NaN,Der Karpatenhund,NaN,https://www.rocky-beach.com/hoerspiel/folgen/0...
3,004,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,NaN,Heikedine Körting,NaN,NaN,NaN,NaN,NaN,Die schwarze Katze,NaN,https://www.rocky-beach.com/hoerspiel/folgen/0...
4,005,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,NaN,Heikedine Körting,NaN,NaN,NaN,NaN,NaN,Der Fluch des Rubins,NaN,https://www.rocky-beach.com/hoerspiel/folgen/0...


#### Derive variables

In [6]:
#add book, cover, music, director, editor and sound columns
meta = meta.reindex(meta.columns.tolist() + ["Book", "Cover", "Music", "Director", "Editor", "Sounds"], axis=1) 
def combine_col(df, new_col, columns):  #define function to combine columns
    df[new_col] = df[columns].apply(lambda x: "-".join(x.dropna().str.encode("utf8").astype(str)), axis=1)
combine_col(meta, ["Book"], ["Buch", "Buch und Effekte", "Buch und Redaktion", 
                             u"Hörspielbearbeitung", "Effekte und Redaktion"] ) 
combine_col(meta, ["Cover"], ["Cover-Illustration", "Coverillustration"]) 
combine_col(meta, ["Music"], ["Musik", "Musik und Effekte"]) 
combine_col(meta, ["Director"], ["Regie", "Produktion und Regie", "Regie und Produktion"]) 
combine_col(meta, ["Editor"], ["Redaktion", u"Redaktion und Geräusche"]) 
combine_col(meta, ["Sounds"],[ u'Geräusche', u'Ger�usche']) 

#convert date column
meta["Date"] = pd.to_datetime(meta["Erscheinungsdatum"], format='%d.%m.%Y')

# Extract minutes and return mean and total minutes by episode (mean = total for episodes with only 1 part)
meta["Minutes"] = meta["Laufzeit"].apply(lambda x: re.search("ca.(.*?)in", x).group(1))
meta["Minutes"] = meta["Minutes"].str.replace("M", "").str.replace("m", "").str.replace(" ", "").str.split("+")
meta["Minutes"] = [[int(x) for x in sublist] for sublist in meta["Minutes"]]
meta["Minutes_mean"] = meta["Minutes"].apply(lambda x: sum(x)/len(x))
meta["Minutes_total"] = meta["Minutes"].apply(lambda x: sum(x))

#calculate how many parts the episode has by using the sets of minutes given
meta["Parts"] = meta["Minutes"].apply(lambda x: len(x)) 

In [7]:
meta.head()

,ID,"""Anudhara"" aufgenommen bei Bantree Records, Worms / Produzent",Buch,Buch und Effekte,Buch und Redaktion,Cover-Illustration,Coverillustration,Design,Effekte und Redaktion,Erscheinungsdatum,...,Cover,Music,Director,Editor,Sounds,Date,Minutes,Minutes_mean,Minutes_total,Parts
0,001,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,,B. Brac/B. George,Heikedine Körting,,,1979-10-12,[46],46,46,1
1,002,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,,B. Brac/B. George,Heikedine Körting,,,1979-10-12,[46],46,46,1
2,003,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,,B. Brac/B. George,Heikedine Körting,,,1979-10-12,[44],44,44,1
3,004,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,,B. Brac/B. George,Heikedine Körting,,,1979-10-12,[43],43,43,1
4,005,NaN,H. G. Francis,NaN,NaN,NaN,NaN,NaN,NaN,12.10.1979,...,,B. Brac/B. George,Heikedine Körting,,,1979-10-12,[47],47,47,1


#### Save output

In [8]:
meta.to_csv((os.path.join(folder, "meta.csv")), encoding='utf8', index=False)

### Actors

In [113]:
actor_all = []

for i in range(len(html_list)):
    actor = pd.read_html(html_list[i])[5].dropna()
    actor["ID"] = episode[i]
    actor_all.append(actor)
     
actor_all = pd.concat(actor_all, axis=0, sort=True)  #join all individual tables together

print(actor_all.shape)
print("")
print(actor_all.head())
print("")

(2804, 5)

                                    0                 1    2    3   ID
0                 Hitchcock, Erzähler     Peter Pasetti  NaN  NaN  001
1       Justus Jonas, Erster Detektiv   Oliver Rohrbeck  NaN  NaN  001
2        Peter Shaw, Zweiter Detektiv    Jens Wawrczeck  NaN  NaN  001
3  Bob Andrews, Recherchen und Archiv  Andreas Fröhlich  NaN  NaN  001
4        Mr. Fentriss, Schriftsteller   Richard Lauffen  NaN  NaN  001



In [114]:
#clean data
#inspect rows with entries in column 3 and 4
print(actor_all.loc[actor_all[2].isna()== False])

#insert wrong data into new dataframe
actors_wrong = pd.DataFrame([[actor_all.loc[actor_all[2].isna()== False, 2].item(), 
                             actor_all.loc[actor_all[2].isna()== False, 3].item(),
                            actor_all.loc[actor_all[2].isna()== False, "ID"].item()]], columns=[0,1,"ID"])

#drop empty columns
actor_all = actor_all.drop([2,3], axis=1)

#append incorrect actors
actor_all = actor_all.append(actors_wrong)

#rename columns
actor_all = actor_all.rename(columns={0:"Role", 1:"Actor"})

             0                                        1          2  \
11  Alvin Cray  Michael Bideller Mr. LloydLutz Mackensy  Mr. Lloyd   

                3   ID  
11  Lutz Mackensy  173  


In [115]:
#derive clean name columns

#get real name not the pseudonym (if they have one)
actor_all["Actor_clean"] = actor_all["Actor"].str.split("[").str[-1].str.replace("]", "")

#aristocratic or academic title
actor_all["Title"] = 0
actor_all.loc[actor_all["Actor_clean"].str.contains(" van "), "Title"] = 1
actor_all.loc[actor_all["Actor_clean"].str.contains(" von "), "Title"] = 1
actor_all.loc[actor_all["Actor_clean"].str.contains("Dr. "), "Title"] = 1
actor_all.loc[actor_all["Actor_clean"].str.contains("Prof. "), "Title"] = 1

#remove aristocratic title in cleaned name
actor_all["Actor_clean"] = (actor_all["Actor_clean"].str.replace(" von", "")
                            .str.replace(" van", "")
                            .str.replace(" der", "")
                            .str.replace(" de", "")
                            .str.replace("Dr. ", "")
                           .str.replace("Prof. ", ""))

#get first and last name
actor_all["Firstname"] = actor_all["Actor_clean"].str.split(" ").str[0]
actor_all["Surename"] = actor_all["Actor_clean"].str.split(" ").str[-1]

#get number of names
actor_all["No_names"] = actor_all["Actor_clean"].str.split(" ").apply(lambda x: len(x))
actor_all.head()

,Role,Actor,ID,Actor_clean,Title,Firstname,Surename,No_names
0,"Hitchcock, Erzähler",Peter Pasetti,001,Peter Pasetti,0,Peter,Pasetti,2
1,"Justus Jonas, Erster Detektiv",Oliver Rohrbeck,001,Oliver Rohrbeck,0,Oliver,Rohrbeck,2
2,"Peter Shaw, Zweiter Detektiv",Jens Wawrczeck,001,Jens Wawrczeck,0,Jens,Wawrczeck,2
3,"Bob Andrews, Recherchen und Archiv",Andreas Fröhlich,001,Andreas Fröhlich,0,Andreas,Fröhlich,2
4,"Mr. Fentriss, Schriftsteller",Richard Lauffen,001,Richard Lauffen,0,Richard,Lauffen,2


In [123]:
#clean up roles
actor_all.loc[actor_all["Role"].str.contains("Justus Jonas"), "Role"] = 'Justus Jonas'
actor_all.loc[actor_all["Role"].str.contains("Peter Shaw"), "Role"] = 'Peter Shaw'
actor_all.loc[actor_all["Role"].str.contains("Bob Andrews"), "Role"] = 'Bob Andrews'
actor_all.loc[actor_all["Role"].str.contains("Hitchcock"), "Role"] = 'Hitchcock'
actor_all.loc[actor_all["Role"].str.contains("Tante Mathilda"), "Role"] = 'Tante Mathilda'
actor_all.loc[actor_all["Role"].str.contains("Onkel Titus"), "Role"] = 'Onkel Titus'
actor_all.loc[actor_all["Role"].str.contains("missar Reynolds"), "Role"] = 'Kommissar Reynolds'
actor_all.loc[actor_all["Role"].str.contains("Skinny"), "Role"] = 'Skinny Norris'
actor_all.loc[actor_all["Role"].str.contains("Godween"), "Role"] = 'Goodween'
actor_all.loc[actor_all["Role"].str.contains("Mr Shaw"), "Role"] = 'Mr. Shaw'
actor_all.loc[actor_all["Role"].str.contains("Herr Andrews"), "Role"] = 'Mr. Andrews'
actor_all.loc[actor_all["Role"].str.contains("Morton"), "Role"] = 'Morton'
actor_all["Role"].str.replace("Mrs ", "Mrs. ").str.replace("Mr ", "Mr. ")
actor_all.loc[actor_all["Role"].str.contains("Inspektor Cottahol"), "Role"] = 'Inspektor Cotta'
actor_all.loc[actor_all["Role"].str.contains("Cotta, Polizeiinspektor"), "Role"] = 'Inspektor Cotta'

#create filter for main actor
actor_all["Main_actor"] = 0
actor_all.loc[(actor_all["Role"]=='Justus Jonas') | (actor_all["Role"]=='Peter Shaw') | (actor_all["Role"]=='Bob Andrews'), "Main_actor"] = 1

#create filter for narrator
actor_all["Narrator"] = 0
actor_all.loc[(actor_all["Role"]=='Hitchcock') | (actor_all["Role"]==u'Erzähler'), "Narrator"] = 1

actor_all.head()

,Role,Actor,ID,Actor_clean,Title,Firstname,Surename,No_names,Gender_count,Gender,Gender_probability
0,Hitchcock,Peter Pasetti,001,Peter Pasetti,0,Peter,Pasetti,2,165452,male,0.99
1,Justus Jonas,Oliver Rohrbeck,001,Oliver Rohrbeck,0,Oliver,Rohrbeck,2,32776,male,0.99
2,Peter Shaw,Jens Wawrczeck,001,Jens Wawrczeck,0,Jens,Wawrczeck,2,23807,male,0.99
3,Bob Andrews,Andreas Fröhlich,001,Andreas Fröhlich,0,Andreas,Fröhlich,2,71365,male,0.99
4,"Mr. Fentriss, Schriftsteller",Richard Lauffen,001,Richard Lauffen,0,Richard,Lauffen,2,101928,male,0.99


In [31]:
#get gender for list of firstnames from Genderize.io

#get unique list of first names
names_unique = list (set (actor_all["Firstname"]))

#access genderize.io api using this repository: tps://github.com/SteelPangolin/genderize
from genderize import Genderize
gender_raw = Genderize().get(names_unique)

gender = []
for i in range(len(gender_raw)):
    c = pd.DataFrame(gender_raw[i].items())[1]
    gender.append(c)

gender = pd.concat(gender, axis=1).transpose().rename(columns={0:"Gender_count", 1:"Gender", 2:"Name", 3:"Gender_probability"})  
gender.head()

,Gender_count,Gender,Name,Gender_probability
1,71365,male,Andreas,0.99
1,17180,male,Mathias,0.99
1,6182,male,Hendrik,0.99
1,6762,male,Harald,0.99
1,32701,female,Brigitte,0.98


In [119]:
# join tables
actor_all = pd.merge(actor_all, gender, left_on='Firstname', right_on = "Name", how="left")
actor_all = actor_all.drop("Name", axis=1)
actor_all.head()

,Role,Actor,ID,Actor_clean,Title,Firstname,Surename,No_names,Gender_count,Gender,Gender_probability
0,Hitchcock,Peter Pasetti,001,Peter Pasetti,0,Peter,Pasetti,2,165452,male,0.99
1,Justus Jonas,Oliver Rohrbeck,001,Oliver Rohrbeck,0,Oliver,Rohrbeck,2,32776,male,0.99
2,Peter Shaw,Jens Wawrczeck,001,Jens Wawrczeck,0,Jens,Wawrczeck,2,23807,male,0.99
3,Bob Andrews,Andreas Fröhlich,001,Andreas Fröhlich,0,Andreas,Fröhlich,2,71365,male,0.99
4,"Mr. Fentriss, Schriftsteller",Richard Lauffen,001,Richard Lauffen,0,Richard,Lauffen,2,101928,male,0.99


In [150]:
#Save output
actor_all.to_csv((os.path.join(folder, "actor_all.csv")), encoding='utf8', index=False)

### Content

In [63]:
content_all = []

for i in range(len(html_list)):
    content = pd.DataFrame(pd.read_html(html_list[i])[3].dropna(how='all').dropna(1).iloc[1])
    content["ID"] = episode[i]
    meta["Source"] = "Rocky-Beach.com"
    content_all.append(content)
    
content_all = pd.concat(content_all, axis=0).rename(columns={1:"content"}) #join all individual objects together
content_all.head()   

,content,ID
0,Der neueste Auftrag an die drei Detektive hört...,001
0,Welches Geheimnis verbirgt sich in einem vergi...,002
0,"""Bei mir spukt es!"" Mit diesem verzweifelten A...",003
0,In einem kleinen Wanderzirkus wittern die drei...,004
0,Alfred Hitchcock und die drei Detektive (Firme...,005


In [64]:
#save output
content_all.to_csv((os.path.join(folder, "content_all.csv")), encoding='utf8', index=False)

### Ratings

In [151]:
#https://www.rocky-beach.com/php/project/f_ausgabe.html   
rating1 = pd.read_html("https://www.rocky-beach.com/php/project/f_ausgabe.html", header=[0])[3] 
rating1 = rating1.dropna(how='all', axis=1).rename(columns={"Folge": "Episode","Unnamed: 4": "Rating", "Stimmen":"Ranking", "Unnamed: 7":"Votes"})
rating1 = rating1.dropna(how='all')
rating1["Episode"] = rating1["Episode"].str.encode("utf8") #clean formatting of episode
rating1["ID"] = ["%03d" % number for number in 
                 (rating1["Episode"].apply(lambda x: re.search(" \((.*?)\)", x).group(1))).astype(float)]
rating1["Episode"] = rating1["Episode"].apply(lambda x: re.search("(.*?)\(", x).group(1))
rating1["Source"] = "Rocky-Beach.com"
rating1

,Episode,Rating,Ranking,Votes,ID,Source
0,Der Super-Papagei,1.6162,4.0,753.0,001,Rocky-Beach.com
1,Der Phantomsee,1.8150,19.0,681.0,002,Rocky-Beach.com
2,Der Karpatenhund,1.6633,8.0,689.0,003,Rocky-Beach.com
3,Die schwarze Katze,2.4022,104.0,644.0,004,Rocky-Beach.com
4,Der Fluch des Rubins,1.8202,20.0,645.0,005,Rocky-Beach.com
5,Der sprechende Totenkopf,1.7682,13.0,647.0,006,Rocky-Beach.com
6,Der unheimliche Drache,2.2975,82.0,605.0,007,Rocky-Beach.com
7,Der grüne Geist,1.8660,25.0,627.0,008,Rocky-Beach.com
8,Die rätselhaften Bilder,2.0443,40.0,609.0,009,Rocky-Beach.com
9,Die flüsternde Mumie,1.8866,26.0,644.0,010,Rocky-Beach.com


## Analysis

In [210]:
## calculate gender roles by episode

no_main = (actor_all["Narrator"] == 0) & (actor_all["Main_actor"] == 0)

gender_balance = actor_all.loc[no_main].groupby(["ID", "Gender"])["Role"].count()
gender_balance = pd.DataFrame(gender_balance).unstack().fillna(0).reset_index("ID")
gender_balance.columns = gender_balance.columns.droplevel(level=0)
gender_balance["total"] = gender_balance[["female", "male"]].sum(axis=1)
gender_balance["female_ratio"] = gender_balance["female"] / gender_balance["total"]
gender_balance = gender_balance.rename(columns={"": "ID"})
gender_balance
gender_balance.columns
#gender_balance["female_ratio"].describe()

Index([u'ID', u'female', u'male', u'total', u'female_ratio'], dtype='object', name=u'Gender')

In [184]:
gender_balance.index.name = None 

In [207]:
gender_balance.index[3]

3

In [ ]:


plt.scatter(x, y, s=area, c=colors, alpha=0.5

In [133]:
#big function
rating1["Folge"] = rating1["Folge"].str.encode("utf8")
id_list = []
for i in range(len(rating1["Folge"])):
    pattern = " \((.*?)\)"
    number = re.search(pattern, rating1["Folge"][i]).group(1)
    id_list.append(number)
id_list

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '1

## 3fragezeichen.de

### Setup list of webpages to scrape

In [7]:
# generate list with all correct links
html_list2 = []
website2 = "https://www.3fragezeichen.net/folgendb.php?nr="
for i in range(1,203):
    html = "%s%s" % (website2, i)
    html_list2.append(html)

html_list2

['https://www.3fragezeichen.net/folgendb.php?nr=1',
 'https://www.3fragezeichen.net/folgendb.php?nr=2',
 'https://www.3fragezeichen.net/folgendb.php?nr=3',
 'https://www.3fragezeichen.net/folgendb.php?nr=4',
 'https://www.3fragezeichen.net/folgendb.php?nr=5',
 'https://www.3fragezeichen.net/folgendb.php?nr=6',
 'https://www.3fragezeichen.net/folgendb.php?nr=7',
 'https://www.3fragezeichen.net/folgendb.php?nr=8',
 'https://www.3fragezeichen.net/folgendb.php?nr=9',
 'https://www.3fragezeichen.net/folgendb.php?nr=10',
 'https://www.3fragezeichen.net/folgendb.php?nr=11',
 'https://www.3fragezeichen.net/folgendb.php?nr=12',
 'https://www.3fragezeichen.net/folgendb.php?nr=13',
 'https://www.3fragezeichen.net/folgendb.php?nr=14',
 'https://www.3fragezeichen.net/folgendb.php?nr=15',
 'https://www.3fragezeichen.net/folgendb.php?nr=16',
 'https://www.3fragezeichen.net/folgendb.php?nr=17',
 'https://www.3fragezeichen.net/folgendb.php?nr=18',
 'https://www.3fragezeichen.net/folgendb.php?nr=19',
 '

### Ratings

In [135]:
# scrape ratings
rows = BeautifulSoup(urlopen(html_list2[28]).read(), 'html.parser').find_all(color="red")
results = []
for row in rows:
    results.append(row.get_text())
#results.append(0)
results

[]

In [362]:
# scrape ratings
results_50 = []
for i in range(50):
    rows = BeautifulSoup(urlopen(html_list2[i]).read(), 'html.parser').find_all(color="red")
    episodes = []
    episodes.append(html_list2[i])
    for row in rows:
        episodes.append(row.get_text())
    results_50.append(episodes)

In [364]:
# scrape ratings for 51-100 pages
results_100 = []
for i in range(50, 100):
    rows = BeautifulSoup(urlopen(html_list2[i]).read(), 'html.parser').find_all(color="red")
    episodes = []
    episodes.append(html_list2[i])
    for row in rows:
        episodes.append(row.get_text())
    results_100.append(episodes)

In [365]:
# scrape ratings for 101-150 pages
results_150 = []
for i in range(100, 150):
    rows = BeautifulSoup(urlopen(html_list2[i]).read(), 'html.parser').find_all(color="red")
    episodes = []
    episodes.append(html_list2[i])
    for row in rows:
        episodes.append(row.get_text())
    results_150.append(episodes)

In [367]:
# scrape ratings for 151 - 202 pages
results_202 = []
for i in range(150, 202):
    rows = BeautifulSoup(urlopen(html_list2[i]).read(), 'html.parser').find_all(color="red")
    episodes = []
    episodes.append(html_list2[i])
    for row in rows:
        episodes.append(row.get_text())
    results_202.append(episodes)

In [382]:
results = pd.concat([pd.DataFrame(results_50), 
                     pd.DataFrame(results_100), 
                     pd.DataFrame(results_150), 
                     pd.DataFrame(results_202)])

results = results.rename(columns ={1: "rating_raw", 2: "fanrating_raw", 0: "html"})
results["rating"] = results["rating_raw"].str.len()
results["fanrating"] = results["fanrating_raw"].str.len()
results["ID"] = ["%03d" % number for number in results["html"].str.extract(r'[=](\d+)', expand=False).astype(int)]

results.head()

,html,rating_raw,fanrating_raw,rating,fanrating,ID
0,https://www.3fragezeichen.net/folgendb.php?nr=1,?????,♥♥♥♥,5.0,4.0,001
1,https://www.3fragezeichen.net/folgendb.php?nr=2,?????,♥♥♥♥,5.0,4.0,002
2,https://www.3fragezeichen.net/folgendb.php?nr=3,?????,♥♥♥♥♥,5.0,5.0,003
3,https://www.3fragezeichen.net/folgendb.php?nr=4,????,♥♥♥♥,4.0,4.0,004
4,https://www.3fragezeichen.net/folgendb.php?nr=5,?????,♥♥♥♥,5.0,4.0,005


In [246]:
# save as csv
results.to_csv((os.path.join(folder, "ratings_fragezeichen_de.csv")), encoding='utf8', index=False)

# Analysis

### Gender balance

In [152]:
## calculate gender roles by episode

no_main =  

actor_all.groupby(["ID", "Gender"])["Role"].count()



ID   Gender
001  female     2
     male      10
002  female     3
     male      11
003  female     1
     male      11
004  male      15
005  female     3
     male      10
006  female     2
     male      13
007  male       9
008  female     2
     male      13
009  female     2
     male      12
010  female     1
     male      13
011  female     1
     male       8
012  female     4
     male      12
013  female     2
     male      13
014  female     2
     male      12
015  female     1
     male      13
016  female     2
     male      10
               ..
190  female     2
     male      10
191  female     3
     male       8
192  female     3
     male      11
193  female     4
     male      15
194  female     8
     male       9
195  female     3
     male       9
196  female     4
     male       9
197  female     6
     male      13
198  female     6
     male      13
199  female     3
     male      10
200  female     5
     male      25
201  female     5
     male       

### Over time

In [515]:
timeline = pd.DataFrame(range(1979, 2021), columns = ["Year"]).astype(str)
timeline.head()

,Year
0,1979
1,1980
2,1981
3,1982
4,1983


In [532]:
years = meta.groupby("Year")[["Parts", "Minutes_mean", "ID"]].agg({
    "Parts": ["sum"], "Minutes_mean" : ["mean", "min", "max"], "ID": ["count","first", "last"]})
years.columns = ["_".join(x) for x in years.columns.ravel()]
years = years.reset_index().rename(columns={"Minutes_mean_mean": "Minutes_mean",
                              "Minutes_mean_min": "Minutes_min", "Minutes_mean_max":"Minutes_max"}) 
years.head()

,Year,Minutes_mean,Minutes_min,Minutes_max,Parts_sum,ID_count,ID_first,ID_last
0,1979.0,45.555556,43,50,9,9,001,009
1,1980.0,42.416667,38,49,12,12,010,021
2,1981.0,43.666667,41,49,6,6,022,027
3,1982.0,42.000000,40,44,2,2,028,030
4,1983.0,43.666667,40,49,3,3,031,033


# rubbish code

## Scrape meta data using soup

In [ ]:
soup = BeautifulSoup(urlopen(html_list[0]).read(), 'html.parser')

In [13]:
# scrape meta data

rows = soup.find_all(class_="gelb")
results = []
for row in rows:
    results.append(row.get_text())
results.append(0)
results
rows

[<span class="gelb">Erz\xe4hlt von:</span>,
 <span class="gelb">Buch:</span>,
 <span class="gelb">Regie:</span>,
 <span class="gelb">Musik:</span>,
 <span class="gelb">Erscheinungsdatum:</span>,
 <span class="gelb">Laufzeit:</span>]

In [6]:
table = soup.find_all('table')[8]

new_table = pd.DataFrame(columns=range(0,2), index = [0]) # I know the size
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
            new_table.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        
new_table

,0,1
0,,


In [502]:
table = soup.find_all('table')[8]
table

<table border="0" cols="2" width="100%">\n<tr>\n<td><span class="gelb">Erz\xe4hlt von:</span></td>\n<td><a href="http://www.rocky-beach.com/special/autorenliste/autorenliste.html?author=2">Robert Arthur</a></td>\n</tr>\n<tr>\n<td><span class="gelb">Buch:</span></td>\n<td>H. G. Francis</td>\n</tr>\n<tr>\n<td><span class="gelb">Regie:</span></td>\n<td>Heikedine K\xf6rting</td>\n</tr>\n<tr>\n<td><span class="gelb">Musik:</span></td>\n<td>B. Brac/B. George</td>\n</tr>\n<tr>\n<td><span class="gelb">Erscheinungsdatum:</span></td>\n<td>12.10.1979</td>\n</tr>\n<tr>\n<td><span class="gelb">Laufzeit:</span></td>\n<td>ca. 46 min.</td>\n</tr>\n<tr>\n<td height="25"></td>\n<td></td>\n</tr>\n<tr>\n<td></td>\n<td><span class="white">H\xf6rspielskript:</span></td>\n</tr>\n<tr valign="CENTER">\n<td height="35" valign="CENTER">\n</td>\n<td valign="CENTER"><a href="../skript/skript_001.pdf" target="01"><img border="0" height="25" nosave="" src="../icons/buch.jpg" width="54"/></a>\n123 kB</td>\n</tr>\n<tr